In [1]:
import pandas as pd
import numpy as np
import pymc as pm
import seaborn as sns
from sklearn.model_selection import train_test_split
pd.set_option('display.max_rows', 500)
import arviz as az
import matplotlib.pyplot as plt

WARNING (aesara.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (aesara.configdefaults): g++ not detected!  Aesara will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set Aesara flags cxx to an empty string.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## READ IN DATA

In [3]:
# select all the players besides dbs and wrs
players = pd.read_csv("https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/players.csv")

positions = ['DE', 'OLB','DT', 'ILB', 'NT', 'MLB', 'LB', 'RB', 'T', 'TE','G','QB','C','FB']
positions_df = players[players['officialPosition'].isin(positions)]
players_list = positions_df['nflId']

In [4]:
# Read in all the weeks but only for the positons above
locations = pd.DataFrame()
for i in range(1,8):
    url = 'https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/week'+str(i)+'.csv'
    week_data = pd.read_csv(url)
    week_data = week_data[(week_data['nflId'].isin(players_list)) | (week_data['team'] == 'football')]
    locations = pd.concat([locations, week_data])

In [5]:
locations

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event
0,2021090900,97,25511.0,1,2021-09-10 00:26:31,12.0,TB,right,37.77,24.22,0.29,0.30,0.03,165.16,84.99,None
1,2021090900,97,25511.0,2,2021-09-10 00:26:31,12.0,TB,right,37.78,24.22,0.23,0.11,0.02,164.33,92.87,None
2,2021090900,97,25511.0,3,2021-09-10 00:26:31,12.0,TB,right,37.78,24.24,0.16,0.10,0.01,160.24,68.55,None
3,2021090900,97,25511.0,4,2021-09-10 00:26:31,12.0,TB,right,37.73,24.25,0.15,0.24,0.06,152.13,296.85,None
4,2021090900,97,25511.0,5,2021-09-10 00:26:31,12.0,TB,right,37.69,24.26,0.25,0.18,0.04,148.33,287.55,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906287,2021102500,3998,NaN,71,2021-10-26 03:17:05,NaN,football,left,112.93,14.52,22.04,0.44,2.62,NaN,NaN,pass_forward
906288,2021102500,3998,NaN,72,2021-10-26 03:17:06,NaN,football,left,111.34,16.07,21.95,1.40,2.22,NaN,NaN,None
906289,2021102500,3998,NaN,73,2021-10-26 03:17:06,NaN,football,left,109.74,17.60,21.81,2.02,2.21,NaN,NaN,None
906290,2021102500,3998,NaN,74,2021-10-26 03:17:06,NaN,football,left,108.17,19.12,21.66,2.39,2.19,NaN,NaN,None


In [6]:
team_list = locations.drop_duplicates(subset = ['nflId'])[['nflId','team']]

In [7]:
player = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/players.csv')

In [8]:
players = player[['displayName','nflId']].rename(columns = {"displayName":"tackle_name"})

In [12]:
player_teams = team_list.merge(players, on = 'nflId', how = 'left')

In [14]:
#player_teams.to_csv('player_teams.csv', index = False)

In [149]:
pff = pd.read_csv("https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/pffScoutingData.csv")

In [150]:
pff

,gameId,playId,nflId,pff_role,pff_positionLinedUp,pff_hit,pff_hurry,pff_sack,pff_beatenByDefender,pff_hitAllowed,pff_hurryAllowed,pff_sackAllowed,pff_nflIdBlockedPlayer,pff_blockType,pff_backFieldBlock
0,2021090900,97,25511,Pass,QB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021090900,97,35481,Pass Route,TE-L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021090900,97,35634,Pass Route,LWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021090900,97,39985,Pass Route,HB-R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021090900,97,40151,Pass Block,C,NaN,NaN,NaN,0.0,0.0,0.0,0.0,44955.0,SW,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188249,2021110100,4433,52507,Pass Block,LT,NaN,NaN,NaN,0.0,0.0,1.0,0.0,43338.0,PP,0.0
188250,2021110100,4433,52546,Coverage,SCBoR,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188251,2021110100,4433,52573,Pass Route,SLoWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188252,2021110100,4433,52585,Pass Rush,LEO,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
# Going to join these columns in with the tracking data
pff_filter = pff[['gameId','playId','nflId','pff_role','pff_positionLinedUp','pff_nflIdBlockedPlayer','pff_blockType']]

In [152]:
locations_ball = locations.merge(pff_filter, on=['gameId','playId','nflId'], how='left')

In [153]:
plays = pd.read_csv("https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/plays.csv")

In [154]:
plays

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,...,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType
0,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep...,1,3,2,TB,DAL,TB,33,...,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man
1,2021090900,137,(13:18) (Shotgun) D.Prescott pass deep left to...,1,1,10,DAL,TB,DAL,2,...,NaN,108.0,EMPTY,"1 RB, 2 TE, 2 WR",6.0,"4 DL, 4 LB, 3 DB",TRADITIONAL,0,Cover-3,Zone
2,2021090900,187,(12:23) (Shotgun) D.Prescott pass short middle...,1,2,6,DAL,TB,DAL,34,...,NaN,76.0,SHOTGUN,"0 RB, 2 TE, 3 WR",6.0,"3 DL, 3 LB, 5 DB",TRADITIONAL,0,Cover-3,Zone
3,2021090900,282,(9:56) D.Prescott pass incomplete deep left to...,1,1,10,DAL,TB,TB,39,...,NaN,49.0,SINGLEBACK,"1 RB, 2 TE, 2 WR",6.0,"4 DL, 3 LB, 4 DB",TRADITIONAL,1,Cover-3,Zone
4,2021090900,349,(9:46) (Shotgun) D.Prescott pass incomplete sh...,1,3,15,DAL,TB,TB,44,...,NaN,54.0,SHOTGUN,"1 RB, 1 TE, 3 WR",7.0,"3 DL, 4 LB, 4 DB",TRADITIONAL,0,Cover-3,Zone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8553,2021110100,4310,(1:56) (Shotgun) P.Mahomes sacked at NYG 16 fo...,4,3,8,KC,NYG,NYG,8,...,NaN,18.0,SHOTGUN,"1 RB, 1 TE, 3 WR",4.0,"1 DL, 3 LB, 7 DB",SCRAMBLE,0,Bracket,Other
8554,2021110100,4363,(1:07) (Shotgun) D.Jones pass short right to E...,4,1,10,NYG,KC,NYG,25,...,NaN,35.0,SHOTGUN,"1 RB, 1 TE, 3 WR",5.0,"4 DL, 1 LB, 6 DB",SCRAMBLE,0,Cover-2,Zone
8555,2021110100,4392,"(1:01) (No Huddle, Shotgun) D.Jones sacked at ...",4,2,7,NYG,KC,NYG,28,...,NaN,38.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 1 LB, 6 DB",TRADITIONAL,0,Cover-2,Zone
8556,2021110100,4411,"(:39) (No Huddle, Shotgun) D.Jones pass incomp...",4,3,15,NYG,KC,NYG,20,...,NaN,30.0,SHOTGUN,"1 RB, 1 TE, 3 WR",5.0,"4 DL, 1 LB, 6 DB",TRADITIONAL,0,Cover-2,Zone


In [155]:
# Also going to be added in to the tracking data
plays_filter = plays[['gameId','playId','dropBackType','pff_playAction']]

In [156]:
locations_ball = locations_ball.merge(plays_filter, on=['gameId','playId'], how='left')

In [157]:
locations_ball

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,...,dis,o,dir,event,pff_role,pff_positionLinedUp,pff_nflIdBlockedPlayer,pff_blockType,dropBackType,pff_playAction
0,2021090900,97,25511.0,1,2021-09-10 00:26:31,12.0,TB,right,37.77,24.22,...,0.03,165.16,84.99,None,Pass,QB,NaN,NaN,TRADITIONAL,0
1,2021090900,97,25511.0,2,2021-09-10 00:26:31,12.0,TB,right,37.78,24.22,...,0.02,164.33,92.87,None,Pass,QB,NaN,NaN,TRADITIONAL,0
2,2021090900,97,25511.0,3,2021-09-10 00:26:31,12.0,TB,right,37.78,24.24,...,0.01,160.24,68.55,None,Pass,QB,NaN,NaN,TRADITIONAL,0
3,2021090900,97,25511.0,4,2021-09-10 00:26:31,12.0,TB,right,37.73,24.25,...,0.06,152.13,296.85,None,Pass,QB,NaN,NaN,TRADITIONAL,0
4,2021090900,97,25511.0,5,2021-09-10 00:26:31,12.0,TB,right,37.69,24.26,...,0.04,148.33,287.55,None,Pass,QB,NaN,NaN,TRADITIONAL,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4863391,2021102500,3998,NaN,71,2021-10-26 03:17:05,NaN,football,left,112.93,14.52,...,2.62,NaN,NaN,pass_forward,NaN,NaN,NaN,NaN,SCRAMBLE,0
4863392,2021102500,3998,NaN,72,2021-10-26 03:17:06,NaN,football,left,111.34,16.07,...,2.22,NaN,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0
4863393,2021102500,3998,NaN,73,2021-10-26 03:17:06,NaN,football,left,109.74,17.60,...,2.21,NaN,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0
4863394,2021102500,3998,NaN,74,2021-10-26 03:17:06,NaN,football,left,108.17,19.12,...,2.19,NaN,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0


In [158]:
# create unique play ID for each play
locations_ball['uniqueplayId'] = locations_ball['gameId'].astype(str) + locations_ball['playId'].astype(str)

In [159]:
# Dropping all of the rows that occur before the snap of the ball
snap_time = locations_ball.loc[locations_ball['event'] == 'ball_snap',['uniqueplayId', 'frameId']].rename(columns = {'frameId':'ball_snap_frame'})
snap_time_unique = snap_time.drop_duplicates(subset = 'uniqueplayId')

In [160]:
locations_ball2 = locations_ball.merge(snap_time_unique, on = 'uniqueplayId', how = 'left')
locations_ball2

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,...,dir,event,pff_role,pff_positionLinedUp,pff_nflIdBlockedPlayer,pff_blockType,dropBackType,pff_playAction,uniqueplayId,ball_snap_frame
0,2021090900,97,25511.0,1,2021-09-10 00:26:31,12.0,TB,right,37.77,24.22,...,84.99,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,6.0
1,2021090900,97,25511.0,2,2021-09-10 00:26:31,12.0,TB,right,37.78,24.22,...,92.87,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,6.0
2,2021090900,97,25511.0,3,2021-09-10 00:26:31,12.0,TB,right,37.78,24.24,...,68.55,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,6.0
3,2021090900,97,25511.0,4,2021-09-10 00:26:31,12.0,TB,right,37.73,24.25,...,296.85,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,6.0
4,2021090900,97,25511.0,5,2021-09-10 00:26:31,12.0,TB,right,37.69,24.26,...,287.55,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4863391,2021102500,3998,NaN,71,2021-10-26 03:17:05,NaN,football,left,112.93,14.52,...,NaN,pass_forward,NaN,NaN,NaN,NaN,SCRAMBLE,0,20211025003998,8.0
4863392,2021102500,3998,NaN,72,2021-10-26 03:17:06,NaN,football,left,111.34,16.07,...,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,20211025003998,8.0
4863393,2021102500,3998,NaN,73,2021-10-26 03:17:06,NaN,football,left,109.74,17.60,...,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,20211025003998,8.0
4863394,2021102500,3998,NaN,74,2021-10-26 03:17:06,NaN,football,left,108.17,19.12,...,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,20211025003998,8.0


In [161]:
locations_ball3 = locations_ball2[~(locations_ball2['frameId'] < locations_ball2['ball_snap_frame'])]

In [162]:
locations_ball3

,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,...,dir,event,pff_role,pff_positionLinedUp,pff_nflIdBlockedPlayer,pff_blockType,dropBackType,pff_playAction,uniqueplayId,ball_snap_frame
5,2021090900,97,25511.0,6,2021-09-10 00:26:31,12.0,TB,right,37.64,24.26,...,282.72,ball_snap,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,6.0
6,2021090900,97,25511.0,7,2021-09-10 00:26:31,12.0,TB,right,37.56,24.26,...,272.95,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,6.0
7,2021090900,97,25511.0,8,2021-09-10 00:26:31,12.0,TB,right,37.47,24.25,...,267.49,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,6.0
8,2021090900,97,25511.0,9,2021-09-10 00:26:31,12.0,TB,right,37.38,24.24,...,263.48,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,6.0
9,2021090900,97,25511.0,10,2021-09-10 00:26:32,12.0,TB,right,37.27,24.23,...,263.77,None,Pass,QB,NaN,NaN,TRADITIONAL,0,202109090097,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4863391,2021102500,3998,NaN,71,2021-10-26 03:17:05,NaN,football,left,112.93,14.52,...,NaN,pass_forward,NaN,NaN,NaN,NaN,SCRAMBLE,0,20211025003998,8.0
4863392,2021102500,3998,NaN,72,2021-10-26 03:17:06,NaN,football,left,111.34,16.07,...,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,20211025003998,8.0
4863393,2021102500,3998,NaN,73,2021-10-26 03:17:06,NaN,football,left,109.74,17.60,...,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,20211025003998,8.0
4863394,2021102500,3998,NaN,74,2021-10-26 03:17:06,NaN,football,left,108.17,19.12,...,NaN,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,20211025003998,8.0


In [163]:
# Add the location of the football on the play to each row then standardize x and y on this location
locations_ball4 = locations_ball3.loc[(locations_ball3['team'] == 'football') & (locations_ball3['event'] == 'ball_snap'),
                               ['uniqueplayId', 'x', 'y']
                               ].rename(columns={'x':'football_x',
                                                 'y':'football_y'}).merge(locations_ball3, on='uniqueplayId', how='left')

locations_ball4['new_x'] = np.where(locations_ball4['playDirection']=='left', 
                                   locations_ball4['football_x']-locations_ball4['x'],
                                   locations_ball4['x']-locations_ball4['football_x'])

locations_ball4['new_y'] = np.where(locations_ball4['playDirection']=='left', 
                                   locations_ball4['football_y']-locations_ball4['y'],
                                   locations_ball4['y']-locations_ball4['football_y'])


In [164]:
locations_ball4

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,event,pff_role,pff_positionLinedUp,pff_nflIdBlockedPlayer,pff_blockType,dropBackType,pff_playAction,ball_snap_frame,new_x,new_y
0,202109090097,41.56,23.92,2021090900,97,25511.0,6,2021-09-10 00:26:31,12.0,TB,...,ball_snap,Pass,QB,NaN,NaN,TRADITIONAL,0,6.0,-3.92,0.34
1,202109090097,41.56,23.92,2021090900,97,25511.0,7,2021-09-10 00:26:31,12.0,TB,...,None,Pass,QB,NaN,NaN,TRADITIONAL,0,6.0,-4.00,0.34
2,202109090097,41.56,23.92,2021090900,97,25511.0,8,2021-09-10 00:26:31,12.0,TB,...,None,Pass,QB,NaN,NaN,TRADITIONAL,0,6.0,-4.09,0.33
3,202109090097,41.56,23.92,2021090900,97,25511.0,9,2021-09-10 00:26:31,12.0,TB,...,None,Pass,QB,NaN,NaN,TRADITIONAL,0,6.0,-4.18,0.32
4,202109090097,41.56,23.92,2021090900,97,25511.0,10,2021-09-10 00:26:32,12.0,TB,...,None,Pass,QB,NaN,NaN,TRADITIONAL,0,6.0,-4.29,0.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4209300,20211025003998,105.09,23.78,2021102500,3998,NaN,71,2021-10-26 03:17:05,NaN,football,...,pass_forward,NaN,NaN,NaN,NaN,SCRAMBLE,0,8.0,-7.84,9.26
4209301,20211025003998,105.09,23.78,2021102500,3998,NaN,72,2021-10-26 03:17:06,NaN,football,...,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,8.0,-6.25,7.71
4209302,20211025003998,105.09,23.78,2021102500,3998,NaN,73,2021-10-26 03:17:06,NaN,football,...,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,8.0,-4.65,6.18
4209303,20211025003998,105.09,23.78,2021102500,3998,NaN,74,2021-10-26 03:17:06,NaN,football,...,None,NaN,NaN,NaN,NaN,SCRAMBLE,0,8.0,-3.08,4.66


In [165]:
## Filtering to only be traditional dropbacks
locations_ball5 = locations_ball4[locations_ball4['dropBackType'] == 'TRADITIONAL']

### REMOVING CHIP BLOCK

In [166]:
locations_ball5['chip_side'] = np.where(locations_ball5['pff_blockType']=='CH', np.where(locations_ball5['new_y']>0, 'right', 'left'), 'none')
locations_ball5['chip_side'] = locations_ball5.groupby('uniqueplayId')['chip_side'].transform(lambda x: 'left' if 'left' in x.unique() else 'right' if 'right' in x.unique() else 'none')

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_78753/296146065.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_ball5['chip_side'] = np.where(locations_ball5['pff_blockType']=='CH', np.where(locations_ball5['new_y']>0, 'right', 'left'), 'none')
/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_78753/296146065.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_ball5['chip_side'] = locations_ball5.groupby('uniqueplayId')['chip_side'].transform(lambda x: 'left' if '

In [167]:
# Remove the offensive linesmen on the side of the field where the chip is 
locations_no_chip = locations_ball5.loc[~(((locations_ball5['pff_positionLinedUp']=='LT')& (locations_ball5['chip_side']=='left'))|
                                ((locations_ball5['pff_positionLinedUp']=='RT')& (locations_ball5['chip_side']=='right')))]

In [168]:
# Take out the rbs, tes, fbs
chip_positions = ['HB-L','HB-R','TE-R','HB','TE-L','TE-oR','TE-iR','TE-iL','TE-oL','FB-R','FB-L','FB']
locations_no_chip = locations_no_chip[~(locations_no_chip['pff_positionLinedUp'].isin(chip_positions))]

In [169]:
## Now going to filter out anything after the ball has been thrown or the qb has been sacked (or quickly scrambled before a sack)
end_time = locations_no_chip.loc[locations_no_chip['event'].isin(['pass_forward','qb_sack','qb_strip_sack', 'autoevent_passforward', 'run']),['uniqueplayId', 'frameId']].rename(columns = {'frameId':'end_frame'}).drop_duplicates()
end_time = end_time.groupby('uniqueplayId')['end_frame'].agg('min').reset_index()

locations_final = locations_no_chip.merge(end_time, on = 'uniqueplayId', how = 'left')
locations_final = locations_final[locations_final['frameId']<=locations_final['end_frame']]

In [170]:
# Left with only plays that resulted in a sack, strip sack, or throw by the qb
locations_final

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,pff_positionLinedUp,pff_nflIdBlockedPlayer,pff_blockType,dropBackType,pff_playAction,ball_snap_frame,new_x,new_y,chip_side,end_frame
0,202109090097,41.56,23.92,2021090900,97,25511.0,6,2021-09-10 00:26:31,12.0,TB,...,QB,NaN,NaN,TRADITIONAL,0,6.0,-3.92,0.34,none,38
1,202109090097,41.56,23.92,2021090900,97,25511.0,7,2021-09-10 00:26:31,12.0,TB,...,QB,NaN,NaN,TRADITIONAL,0,6.0,-4.00,0.34,none,38
2,202109090097,41.56,23.92,2021090900,97,25511.0,8,2021-09-10 00:26:31,12.0,TB,...,QB,NaN,NaN,TRADITIONAL,0,6.0,-4.09,0.33,none,38
3,202109090097,41.56,23.92,2021090900,97,25511.0,9,2021-09-10 00:26:31,12.0,TB,...,QB,NaN,NaN,TRADITIONAL,0,6.0,-4.18,0.32,none,38
4,202109090097,41.56,23.92,2021090900,97,25511.0,10,2021-09-10 00:26:32,12.0,TB,...,QB,NaN,NaN,TRADITIONAL,0,6.0,-4.29,0.31,none,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2613945,20211025003926,85.95,23.68,2021102500,3926,NaN,40,2021-10-26 03:14:08,NaN,football,...,NaN,NaN,NaN,TRADITIONAL,0,7.0,-6.86,-2.19,none,44
2613946,20211025003926,85.95,23.68,2021102500,3926,NaN,41,2021-10-26 03:14:08,NaN,football,...,NaN,NaN,NaN,TRADITIONAL,0,7.0,-6.94,-2.56,none,44
2613947,20211025003926,85.95,23.68,2021102500,3926,NaN,42,2021-10-26 03:14:08,NaN,football,...,NaN,NaN,NaN,TRADITIONAL,0,7.0,-7.03,-2.95,none,44
2613948,20211025003926,85.95,23.68,2021102500,3926,NaN,43,2021-10-26 03:14:08,NaN,football,...,NaN,NaN,NaN,TRADITIONAL,0,7.0,-7.12,-3.29,none,44


In [171]:
# Now going to remove any players that aren't the tackles, defensive ends, qb or ball
subset1 = locations_final[['pff_nflIdBlockedPlayer', 'uniqueplayId','pff_positionLinedUp']]
subset2 = subset1[subset1['pff_positionLinedUp'].isin(['LT','RT'])]

In [172]:
# left here with a list of each who each tackle blocked on each play
subset3 = subset2.drop_duplicates(['uniqueplayId', 'pff_positionLinedUp'])
subset3

,pff_nflIdBlockedPlayer,uniqueplayId,pff_positionLinedUp
114,42403.0,202109090097,LT
342,53441.0,202109090097,RT
556,35454.0,2021090900137,LT
680,40074.0,2021090900137,RT
1104,35454.0,2021090900187,LT
...,...,...,...
2612462,41786.0,20211025003684,RT
2612962,46083.0,20211025003904,LT
2613148,37097.0,20211025003904,RT
2613396,46083.0,20211025003926,LT


In [173]:
left_ends = subset3[subset3['pff_positionLinedUp'] == 'LT'].rename(columns = {'pff_nflIdBlockedPlayer':'Right_end'}).drop(['pff_positionLinedUp'],axis = 1)
left_ends

,Right_end,uniqueplayId
114,42403.0,202109090097
556,35454.0,2021090900137
1104,35454.0,2021090900187
1518,34777.0,2021090900282
2443,53441.0,2021090900410
...,...,...
2611187,52556.0,20211025003536
2611846,52556.0,20211025003660
2612312,52556.0,20211025003684
2612962,46083.0,20211025003904


In [174]:
right_ends = subset3[subset3['pff_positionLinedUp'] == 'RT'].rename(columns = {'pff_nflIdBlockedPlayer':'Left_end'}).drop(['pff_positionLinedUp'],axis = 1)
right_ends

,Left_end,uniqueplayId
342,53441.0,202109090097
680,40074.0,2021090900137
1182,35441.0,2021090900187
1708,40074.0,2021090900282
2098,35441.0,2021090900349
...,...,...
2611358,35562.0,20211025003536
2611942,41786.0,20211025003660
2612462,41786.0,20211025003684
2613148,37097.0,20211025003904


In [175]:
locations_right = locations_final.merge(right_ends, on = 'uniqueplayId', how = 'left')
locations_right

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,pff_nflIdBlockedPlayer,pff_blockType,dropBackType,pff_playAction,ball_snap_frame,new_x,new_y,chip_side,end_frame,Left_end
0,202109090097,41.56,23.92,2021090900,97,25511.0,6,2021-09-10 00:26:31,12.0,TB,...,NaN,NaN,TRADITIONAL,0,6.0,-3.92,0.34,none,38,53441.0
1,202109090097,41.56,23.92,2021090900,97,25511.0,7,2021-09-10 00:26:31,12.0,TB,...,NaN,NaN,TRADITIONAL,0,6.0,-4.00,0.34,none,38,53441.0
2,202109090097,41.56,23.92,2021090900,97,25511.0,8,2021-09-10 00:26:31,12.0,TB,...,NaN,NaN,TRADITIONAL,0,6.0,-4.09,0.33,none,38,53441.0
3,202109090097,41.56,23.92,2021090900,97,25511.0,9,2021-09-10 00:26:31,12.0,TB,...,NaN,NaN,TRADITIONAL,0,6.0,-4.18,0.32,none,38,53441.0
4,202109090097,41.56,23.92,2021090900,97,25511.0,10,2021-09-10 00:26:32,12.0,TB,...,NaN,NaN,TRADITIONAL,0,6.0,-4.29,0.31,none,38,53441.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2224655,20211025003926,85.95,23.68,2021102500,3926,NaN,40,2021-10-26 03:14:08,NaN,football,...,NaN,NaN,TRADITIONAL,0,7.0,-6.86,-2.19,none,44,37097.0
2224656,20211025003926,85.95,23.68,2021102500,3926,NaN,41,2021-10-26 03:14:08,NaN,football,...,NaN,NaN,TRADITIONAL,0,7.0,-6.94,-2.56,none,44,37097.0
2224657,20211025003926,85.95,23.68,2021102500,3926,NaN,42,2021-10-26 03:14:08,NaN,football,...,NaN,NaN,TRADITIONAL,0,7.0,-7.03,-2.95,none,44,37097.0
2224658,20211025003926,85.95,23.68,2021102500,3926,NaN,43,2021-10-26 03:14:08,NaN,football,...,NaN,NaN,TRADITIONAL,0,7.0,-7.12,-3.29,none,44,37097.0


In [176]:
locations_left = locations_right.merge(left_ends, on = 'uniqueplayId', how = 'left')
locations_left['pff_positionLinedUp'] = locations_left['pff_positionLinedUp'].fillna('ball')

In [177]:
locations_non_end = locations_left[locations_left['pff_positionLinedUp'].isin(['LT','RT','QB','ball'])]
locations_end = locations_left[((locations_left['nflId'] == locations_left['Left_end'])|(locations_left['nflId'] == locations_left['Right_end']))]

In [178]:
locations_final2 = locations_end.merge(locations_non_end, how = 'outer')

/Users/joshgen/miniconda3/envs/pymc_env/lib/python3.10/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


In [179]:
locations_final2

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,pff_blockType,dropBackType,pff_playAction,ball_snap_frame,new_x,new_y,chip_side,end_frame,Left_end,Right_end
0,202109090097,41.56,23.92,2021090900,97,42403.0,6,2021-09-10 00:26:31,94.0,DAL,...,NaN,TRADITIONAL,0,6.0,2.34,8.71,none,38,53441.0,42403.0
1,202109090097,41.56,23.92,2021090900,97,42403.0,7,2021-09-10 00:26:31,94.0,DAL,...,NaN,TRADITIONAL,0,6.0,2.27,8.70,none,38,53441.0,42403.0
2,202109090097,41.56,23.92,2021090900,97,42403.0,8,2021-09-10 00:26:31,94.0,DAL,...,NaN,TRADITIONAL,0,6.0,2.12,8.62,none,38,53441.0,42403.0
3,202109090097,41.56,23.92,2021090900,97,42403.0,9,2021-09-10 00:26:31,94.0,DAL,...,NaN,TRADITIONAL,0,6.0,1.93,8.54,none,38,53441.0,42403.0
4,202109090097,41.56,23.92,2021090900,97,42403.0,10,2021-09-10 00:26:32,94.0,DAL,...,NaN,TRADITIONAL,0,6.0,1.55,8.32,none,38,53441.0,42403.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927342,20211025003926,85.95,23.68,2021102500,3926,NaN,40,2021-10-26 03:14:08,NaN,football,...,NaN,TRADITIONAL,0,7.0,-6.86,-2.19,none,44,37097.0,46083.0
927343,20211025003926,85.95,23.68,2021102500,3926,NaN,41,2021-10-26 03:14:08,NaN,football,...,NaN,TRADITIONAL,0,7.0,-6.94,-2.56,none,44,37097.0,46083.0
927344,20211025003926,85.95,23.68,2021102500,3926,NaN,42,2021-10-26 03:14:08,NaN,football,...,NaN,TRADITIONAL,0,7.0,-7.03,-2.95,none,44,37097.0,46083.0
927345,20211025003926,85.95,23.68,2021102500,3926,NaN,43,2021-10-26 03:14:08,NaN,football,...,NaN,TRADITIONAL,0,7.0,-7.12,-3.29,none,44,37097.0,46083.0


In [180]:
# locations_final2.to_csv('data_not_flipped.csv', index = False)

In [181]:
### Next going to want to flip one of the ends to standardize

In [182]:
locations_final2['pff_positionLinedUp'].value_counts()

ball     164231
QB       164231
RT       156450
LT       146663
LEO       66316
REO       60389
ROLB      51363
LOLB      47993
LE        20745
RE        15796
DLT       14288
DRT       13078
LILB       1127
NT          903
LLB         737
RILB        687
NLT         596
NRT         532
RLB         496
SCBR        258
MLB         171
SCBL        158
SCBiL        61
SCBiR        32
RCB          24
SCBoL        22
Name: pff_positionLinedUp, dtype: int64

In [183]:
loc_subset = locations_final2[((locations_final2['event'] == 'ball_snap') & (locations_final2['new_y'] < 0))]

In [184]:
loc_subset2 = loc_subset[['uniqueplayId','nflId']]
loc_subset2['Flip'] = 1

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_78753/1030295974.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loc_subset2['Flip'] = 1


In [185]:
locations_flip = locations_final2.merge(loc_subset2, on = ['uniqueplayId','nflId'], how = 'left')
locations_flip['Flip'] = locations_flip['Flip'].fillna(0)
locations_flip['Flip'].value_counts()

0.0    557950
1.0    369397
Name: Flip, dtype: int64

In [186]:
locations_flip2 = locations_flip[((locations_flip['Flip'] == 1) | locations_flip['pff_positionLinedUp'].isin(['QB','ball']))]
locations_flip2

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_frame,new_x,new_y,chip_side,end_frame,Left_end,Right_end,Flip
33,202109090097,41.56,23.92,2021090900,97,53441.0,6,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,2.12,-1.99,none,38,53441.0,42403.0,1.0
34,202109090097,41.56,23.92,2021090900,97,53441.0,7,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,2.06,-1.96,none,38,53441.0,42403.0,1.0
35,202109090097,41.56,23.92,2021090900,97,53441.0,8,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,1.96,-1.91,none,38,53441.0,42403.0,1.0
36,202109090097,41.56,23.92,2021090900,97,53441.0,9,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,1.80,-1.85,none,38,53441.0,42403.0,1.0
37,202109090097,41.56,23.92,2021090900,97,53441.0,10,2021-09-10 00:26:32,11.0,DAL,...,TRADITIONAL,0,6.0,1.56,-1.72,none,38,53441.0,42403.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927342,20211025003926,85.95,23.68,2021102500,3926,NaN,40,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-6.86,-2.19,none,44,37097.0,46083.0,0.0
927343,20211025003926,85.95,23.68,2021102500,3926,NaN,41,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-6.94,-2.56,none,44,37097.0,46083.0,0.0
927344,20211025003926,85.95,23.68,2021102500,3926,NaN,42,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-7.03,-2.95,none,44,37097.0,46083.0,0.0
927345,20211025003926,85.95,23.68,2021102500,3926,NaN,43,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-7.12,-3.29,none,44,37097.0,46083.0,0.0


In [187]:
locations_flip2['pff_positionLinedUp'] = locations_flip2['pff_positionLinedUp'].replace({'QB': 'QB_flip', 'ball': 'ball_flip'})

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_78753/3487504298.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_flip2['pff_positionLinedUp'] = locations_flip2['pff_positionLinedUp'].replace({'QB': 'QB_flip', 'ball': 'ball_flip'})


In [188]:
locations_flip2['pff_positionLinedUp'].value_counts()

QB_flip      164231
ball_flip    164231
RT           156393
LEO           66287
LOLB          47993
LE            20745
DLT           14288
LILB           1127
LLB             737
NLT             570
NT              333
SCBL            158
MLB              86
SCBiL            61
SCBoL            22
Name: pff_positionLinedUp, dtype: int64

In [189]:
locations_flip2['Flip'] = 1
locations_flip2['new_y'] = locations_flip2['new_y'] * -1

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_78753/3706249968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_flip2['Flip'] = 1
/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_78753/3706249968.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_flip2['new_y'] = locations_flip2['new_y'] * -1


In [190]:
locations_flip2['new_y'].value_counts()

-0.00     13819
 0.02      4139
-0.02      4116
-0.09      3803
-0.07      3580
          ...  
-29.99        1
-29.97        1
-29.93        1
-29.90        1
 10.39        1
Name: new_y, Length: 4509, dtype: int64

In [191]:
locations_same = locations_flip[((locations_flip['Flip'] == 0) | locations_flip['pff_positionLinedUp'].isin(['QB','ball']))]
locations_same['Flip'] = 0

/var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/ipykernel_78753/2767798611.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_same['Flip'] = 0


In [192]:
locations_with_flip = pd.concat([locations_same, locations_flip2])

In [193]:
locations_with_flip

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_frame,new_x,new_y,chip_side,end_frame,Left_end,Right_end,Flip
0,202109090097,41.56,23.92,2021090900,97,42403.0,6,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,6.0,2.34,8.71,none,38,53441.0,42403.0,0
1,202109090097,41.56,23.92,2021090900,97,42403.0,7,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,6.0,2.27,8.70,none,38,53441.0,42403.0,0
2,202109090097,41.56,23.92,2021090900,97,42403.0,8,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,6.0,2.12,8.62,none,38,53441.0,42403.0,0
3,202109090097,41.56,23.92,2021090900,97,42403.0,9,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,6.0,1.93,8.54,none,38,53441.0,42403.0,0
4,202109090097,41.56,23.92,2021090900,97,42403.0,10,2021-09-10 00:26:32,94.0,DAL,...,TRADITIONAL,0,6.0,1.55,8.32,none,38,53441.0,42403.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927342,20211025003926,85.95,23.68,2021102500,3926,NaN,40,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-6.86,2.19,none,44,37097.0,46083.0,1
927343,20211025003926,85.95,23.68,2021102500,3926,NaN,41,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-6.94,2.56,none,44,37097.0,46083.0,1
927344,20211025003926,85.95,23.68,2021102500,3926,NaN,42,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-7.03,2.95,none,44,37097.0,46083.0,1
927345,20211025003926,85.95,23.68,2021102500,3926,NaN,43,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-7.12,3.29,none,44,37097.0,46083.0,1


In [194]:
## Now going to remove any obs where the end dropped to coverage
flipped = locations_with_flip[locations_with_flip['Flip'] == 1]
no_flip = locations_with_flip[locations_with_flip['Flip'] == 0]

In [195]:
flipped_no_coverage = flipped[flipped['nflId'] == flipped['Left_end']]
flipped_no_coverage = flipped_no_coverage[['uniqueplayId','pff_role']]
flipped_rush = flipped_no_coverage[flipped_no_coverage['pff_role'] == 'Pass Rush']
flipped_rush

,uniqueplayId,pff_role
33,202109090097,Pass Rush
34,202109090097,Pass Rush
35,202109090097,Pass Rush
36,202109090097,Pass Rush
37,202109090097,Pass Rush
...,...,...
295729,20211025003926,Pass Rush
295730,20211025003926,Pass Rush
295731,20211025003926,Pass Rush
295732,20211025003926,Pass Rush


In [196]:
same_no_coverage = no_flip[no_flip['nflId'] == no_flip['Right_end']]
same_no_coverage = same_no_coverage[['uniqueplayId','pff_role']]
same_rush = same_no_coverage[same_no_coverage['pff_role'] == 'Pass Rush']
same_rush

,uniqueplayId,pff_role
0,202109090097,Pass Rush
1,202109090097,Pass Rush
2,202109090097,Pass Rush
3,202109090097,Pass Rush
4,202109090097,Pass Rush
...,...,...
295767,20211025003926,Pass Rush
295768,20211025003926,Pass Rush
295769,20211025003926,Pass Rush
295770,20211025003926,Pass Rush


In [197]:
flipped_keep = flipped_rush['uniqueplayId']
flipped2 = flipped[flipped['uniqueplayId'].isin(flipped_keep)]
flipped2

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_frame,new_x,new_y,chip_side,end_frame,Left_end,Right_end,Flip
33,202109090097,41.56,23.92,2021090900,97,53441.0,6,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,2.12,1.99,none,38,53441.0,42403.0,1
34,202109090097,41.56,23.92,2021090900,97,53441.0,7,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,2.06,1.96,none,38,53441.0,42403.0,1
35,202109090097,41.56,23.92,2021090900,97,53441.0,8,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,1.96,1.91,none,38,53441.0,42403.0,1
36,202109090097,41.56,23.92,2021090900,97,53441.0,9,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,1.80,1.85,none,38,53441.0,42403.0,1
37,202109090097,41.56,23.92,2021090900,97,53441.0,10,2021-09-10 00:26:32,11.0,DAL,...,TRADITIONAL,0,6.0,1.56,1.72,none,38,53441.0,42403.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927342,20211025003926,85.95,23.68,2021102500,3926,NaN,40,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-6.86,2.19,none,44,37097.0,46083.0,1
927343,20211025003926,85.95,23.68,2021102500,3926,NaN,41,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-6.94,2.56,none,44,37097.0,46083.0,1
927344,20211025003926,85.95,23.68,2021102500,3926,NaN,42,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-7.03,2.95,none,44,37097.0,46083.0,1
927345,20211025003926,85.95,23.68,2021102500,3926,NaN,43,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-7.12,3.29,none,44,37097.0,46083.0,1


In [198]:
no_flip_keep = same_rush['uniqueplayId']
no_flip2 = no_flip[no_flip['uniqueplayId'].isin(no_flip_keep)]
no_flip2

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_frame,new_x,new_y,chip_side,end_frame,Left_end,Right_end,Flip
0,202109090097,41.56,23.92,2021090900,97,42403.0,6,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,6.0,2.34,8.71,none,38,53441.0,42403.0,0
1,202109090097,41.56,23.92,2021090900,97,42403.0,7,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,6.0,2.27,8.70,none,38,53441.0,42403.0,0
2,202109090097,41.56,23.92,2021090900,97,42403.0,8,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,6.0,2.12,8.62,none,38,53441.0,42403.0,0
3,202109090097,41.56,23.92,2021090900,97,42403.0,9,2021-09-10 00:26:31,94.0,DAL,...,TRADITIONAL,0,6.0,1.93,8.54,none,38,53441.0,42403.0,0
4,202109090097,41.56,23.92,2021090900,97,42403.0,10,2021-09-10 00:26:32,94.0,DAL,...,TRADITIONAL,0,6.0,1.55,8.32,none,38,53441.0,42403.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927342,20211025003926,85.95,23.68,2021102500,3926,NaN,40,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-6.86,-2.19,none,44,37097.0,46083.0,0
927343,20211025003926,85.95,23.68,2021102500,3926,NaN,41,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-6.94,-2.56,none,44,37097.0,46083.0,0
927344,20211025003926,85.95,23.68,2021102500,3926,NaN,42,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-7.03,-2.95,none,44,37097.0,46083.0,0
927345,20211025003926,85.95,23.68,2021102500,3926,NaN,43,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-7.12,-3.29,none,44,37097.0,46083.0,0


In [199]:
locations_with_flip = pd.concat([flipped2, no_flip2])
locations_with_flip

,uniqueplayId,football_x,football_y,gameId,playId,nflId,frameId,time,jerseyNumber,team,...,dropBackType,pff_playAction,ball_snap_frame,new_x,new_y,chip_side,end_frame,Left_end,Right_end,Flip
33,202109090097,41.56,23.92,2021090900,97,53441.0,6,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,2.12,1.99,none,38,53441.0,42403.0,1
34,202109090097,41.56,23.92,2021090900,97,53441.0,7,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,2.06,1.96,none,38,53441.0,42403.0,1
35,202109090097,41.56,23.92,2021090900,97,53441.0,8,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,1.96,1.91,none,38,53441.0,42403.0,1
36,202109090097,41.56,23.92,2021090900,97,53441.0,9,2021-09-10 00:26:31,11.0,DAL,...,TRADITIONAL,0,6.0,1.80,1.85,none,38,53441.0,42403.0,1
37,202109090097,41.56,23.92,2021090900,97,53441.0,10,2021-09-10 00:26:32,11.0,DAL,...,TRADITIONAL,0,6.0,1.56,1.72,none,38,53441.0,42403.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927342,20211025003926,85.95,23.68,2021102500,3926,NaN,40,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-6.86,-2.19,none,44,37097.0,46083.0,0
927343,20211025003926,85.95,23.68,2021102500,3926,NaN,41,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-6.94,-2.56,none,44,37097.0,46083.0,0
927344,20211025003926,85.95,23.68,2021102500,3926,NaN,42,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-7.03,-2.95,none,44,37097.0,46083.0,0
927345,20211025003926,85.95,23.68,2021102500,3926,NaN,43,2021-10-26 03:14:08,NaN,football,...,TRADITIONAL,0,7.0,-7.12,-3.29,none,44,37097.0,46083.0,0


In [200]:
#locations_with_flip.to_csv('data_flipped.csv', index = False)